In [33]:
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox
import pandas as pd
import os
import json

# Sabit Admin bilgileri
ADMIN_CREDENTIALS = {"username": "adu", "password": "1234"}

# Dinamik dosya yolu belirleme
current_dir = os.getcwd()  # Çalıştırılan Python dosyasının bulunduğu dizini al
user_db_file = os.path.join(current_dir, "user_database.json")
file_path = os.path.join(current_dir, "araba_veri_seti.csv")

# Kullanıcı kayıtları
user_database = []

# Araç veri kümesi
try:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    car_data = pd.read_csv(file_path)
    car_data_records = car_data.to_dict(orient="records")  # Veri kümesini sözlük listesine çevir
except Exception as e:
    car_data_records = []  # Eğer dosya yüklenemezse boş bir liste atanır
    print("Error loading dataset:", e)
    
# Değerlendirme geçmişi
valuation_history = []

# Kullanıcı kayıtlarını JSON dosyasına kaydetme fonksiyonu
def save_user_database():
    with open(user_db_file, 'w') as file:
        json.dump(user_database, file)

# JSON dosyasından kullanıcı kayıtlarını okuma fonksiyonu
def load_user_database():
    global user_database
    if os.path.exists(user_db_file):
        with open(user_db_file, 'r') as file:
            user_database = json.load(file)
    else:
        user_database = []

# Program başladığında kullanıcı kayıtlarını yükle
load_user_database()

# Filtreleme fonksiyonu
def apply_filter():
    filtered_cars = car_data_records

    # Filtre kriterlerini al
    marka = marka_combobox.get().lower()
    seri = seri_combobox.get().lower()
    model = model_combobox.get().lower()
    yil = yil_combobox.get()
    renk = renk_combobox.get().lower()
    kasa_tipi = kasa_combobox.get().lower()
    yakit_tipi = yakit_combobox.get().lower()
    vites_tipi = vites_combobox.get().lower()
    kimden = kimden_combobox.get().lower()
    min_fiyat = min_fiyat_entry.get()
    max_fiyat = max_fiyat_entry.get()

    # Filtre uygula
    if marka:
        filtered_cars = [car for car in filtered_cars if marka in str(car.get("Marka", "")).lower()]
    if seri:
        filtered_cars = [car for car in filtered_cars if seri in str(car.get("Seri", "")).lower()]
    if model:
        filtered_cars = [car for car in filtered_cars if model in str(car.get("Model", "")).lower()]
    if yil:
        filtered_cars = [car for car in filtered_cars if yil in str(car.get("Yıl", ""))]
    if renk:
        filtered_cars = [car for car in filtered_cars if renk in str(car.get("Renk", "")).lower()]
    if kasa_tipi:
        filtered_cars = [car for car in filtered_cars if kasa_tipi in str(car.get("Kasa Tipi", "")).lower()]
    if yakit_tipi:
        filtered_cars = [car for car in filtered_cars if yakit_tipi in str(car.get("Yakıt Tipi", "")).lower()]
    if vites_tipi:
        filtered_cars = [car for car in filtered_cars if vites_tipi in str(car.get("Vites Tipi", "")).lower()]
    if kimden:
        filtered_cars = [car for car in filtered_cars if kimden in str(car.get("Kimden", "")).lower()]
    if min_fiyat or max_fiyat:
        try:
            min_fiyat = int(min_fiyat) if min_fiyat else 0
            max_fiyat = int(max_fiyat) if max_fiyat else float("inf")
            filtered_cars = [
                car for car in filtered_cars
                if "Fiyat" in car and pd.notnull(car["Fiyat"]) and min_fiyat <= int(car["Fiyat"]) <= max_fiyat
            ]
        except ValueError:
            pass  # Geçersiz fiyat aralığı girilmişse bu filtre atlanır
        
            # Geçmişe kaydet
    valuation_history.append({
        "criteria": {
            "Marka": marka, "Seri": seri, "Model": model, "Yıl": yil, "Renk": renk,
            "Kasa Tipi": kasa_tipi, "Yakıt Tipi": yakit_tipi, "Vites Tipi": vites_tipi, "Kimden": kimden,
            "Min Fiyat": min_fiyat, "Max Fiyat": max_fiyat
        },
        "results": filtered_cars
    })


    # Sonuçları göster
    search_results.delete(1.0, tk.END)
    if filtered_cars:
        for car in filtered_cars:
            search_results.insert(
                tk.END,
                f"Marka: {car.get('Marka', 'N/A')}, Seri: {car.get('Seri', 'N/A')}, Model: {car.get('Model', 'N/A')}, Yıl: {car.get('Yıl', 'N/A')}, Renk: {car.get('Renk', 'N/A')}, Kasa Tipi: {car.get('Kasa Tipi', 'N/A')}, Yakıt Tipi: {car.get('Yakıt Tipi', 'N/A')}, Vites Tipi: {car.get('Vites Tipi', 'N/A')}, Kimden: {car.get('Kimden', 'N/A')}, Fiyat: {car.get('Fiyat', 'N/A')}$\n"
            )
    else:
        search_results.insert(tk.END, "No cars found.")

# Marka değiştirildiğinde serileri güncelleme
def update_series(event):
    selected_marka = marka_combobox.get().lower()
    related_series = sorted(set(
        str(car["Seri"]) for car in car_data_records
        if "Seri" in car and "Marka" in car and pd.notnull(car["Seri"]) and pd.notnull(car["Marka"]) and selected_marka in car["Marka"].lower()
    ))
    seri_combobox["values"] = related_series
    
def clear_filters():
    marka_combobox.set("")
    seri_combobox.set("")
    model_combobox.set("")
    yil_combobox.set("")
    renk_combobox.set("")
    kasa_combobox.set("")
    yakit_combobox.set("")
    vites_combobox.set("")
    kimden_combobox.set("")
    min_fiyat_entry.delete(0, tk.END)
    max_fiyat_entry.delete(0, tk.END)
    

def show_valuation_history():
    search_results.delete(1.0, tk.END)
    if valuation_history:
        for entry in valuation_history:
            criteria = entry["criteria"]
            results = entry["results"]
            search_results.insert(tk.END, f"Criteria: {criteria}\n")
            for car in results:
                search_results.insert(
                    tk.END,
                    f"  - Marka: {car.get('Marka', 'N/A')}, Model: {car.get('Model', 'N/A')}, Fiyat: {car.get('Fiyat', 'N/A')}\n"
                )
    else:
        search_results.insert(tk.END, "No valuation history available.")
        
        # Çoklu kayıtları saklamak için liste
saved_filter_states = []

def save_filters_with_name():
    # Kullanıcıdan kaydın adını iste
    name = tk.simpledialog.askstring("Save Filter", "Enter a name for this filter:")
    if not name:
        return  # Kullanıcı bir isim vermezse işlem yapılmaz

    # Mevcut filtre durumunu kaydet
    filter_state = {
        "name": name,
        "Marka": marka_combobox.get(),
        "Seri": seri_combobox.get(),
        "Model": model_combobox.get(),
        "Yıl": yil_combobox.get(),
        "Renk": renk_combobox.get(),
        "Kasa Tipi": kasa_combobox.get(),
        "Yakıt Tipi": yakit_combobox.get(),
        "Vites Tipi": vites_combobox.get(),
        "Kimden": kimden_combobox.get(),
        "Min Fiyat": min_fiyat_entry.get(),
        "Max Fiyat": max_fiyat_entry.get()
    }

    # Kaydı listeye ekle
    saved_filter_states.append(filter_state)
    tk.messagebox.showinfo("Success", f"Filter '{name}' saved successfully!")

def load_filter():
    # Kullanıcıdan bir kaydı seçmesini iste
    names = [f["name"] for f in saved_filter_states]
    if not names:
        tk.messagebox.showwarning("No Saved Filters", "No saved filters available.")
        return

    name = tk.simpledialog.askstring("Load Filter", f"Available filters: {', '.join(names)}\nEnter the filter name to load:")
    if not name:
        return

    # İsme göre kaydı bul ve yükle
    selected_filter = next((f for f in saved_filter_states if f["name"] == name), None)
    if selected_filter:
        marka_combobox.set(selected_filter["Marka"])
        seri_combobox.set(selected_filter["Seri"])
        model_combobox.set(selected_filter["Model"])
        yil_combobox.set(selected_filter["Yıl"])
        renk_combobox.set(selected_filter["Renk"])
        kasa_combobox.set(selected_filter["Kasa Tipi"])
        yakit_combobox.set(selected_filter["Yakıt Tipi"])
        vites_combobox.set(selected_filter["Vites Tipi"])
        kimden_combobox.set(selected_filter["Kimden"])
        min_fiyat_entry.delete(0, tk.END)
        min_fiyat_entry.insert(0, selected_filter["Min Fiyat"])
        max_fiyat_entry.delete(0, tk.END)
        max_fiyat_entry.insert(0, selected_filter["Max Fiyat"])
        tk.messagebox.showinfo("Success", f"Filter '{name}' loaded successfully!")
    else:
        tk.messagebox.showerror("Error", f"Filter '{name}' not found.")

def add_home_button(frame):
    tk.Button(frame, text="Anasayfa", command=choose_role_panel, bg="#FFA07A", fg="white").pack(anchor='nw')

# Kullanıcı Ana Menü
def user_main_menu():
    clear_frame()
    tk.Label(root, text="Car Search and Filter", font=("Arial", 14), bg="#1E90FF", fg="white").pack(pady=10)

    global main_frame, marka_combobox, seri_combobox, model_combobox, yil_combobox, renk_combobox, kasa_combobox, sehir_combobox, yakit_combobox, vites_combobox, kimden_combobox, min_fiyat_entry, max_fiyat_entry, search_results

    # Ana çerçeve
    main_frame = tk.Frame(root, bg="#F5F5F5")
    main_frame.pack(fill=tk.BOTH, expand=True)
    add_home_button(main_frame)

    # Sol panel (filtreleme)
    filter_frame = tk.Frame(main_frame, padx=10, pady=10, bg="#F5F5F5")
    filter_frame.pack(side=tk.LEFT, fill=tk.Y)

    # Sağ panel (sonuçlar)
    result_frame = tk.Frame(main_frame, padx=10, pady=10, bg="#F5F5F5")
    result_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

    # Filtreleme alanları
    filter_label = tk.Label(filter_frame, text="Filters", font=("Arial", 14), bg="#F5F5F5")
    filter_label.pack(pady=5)

    tk.Label(filter_frame, text="Marka:", bg="#F5F5F5").pack()
    marka_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Marka"]) for car in car_data_records if "Marka" in car and pd.notnull(car["Marka"])))
    )
    marka_combobox.pack(pady=2)
    marka_combobox.bind("<<ComboboxSelected>>", update_series)  # Marka değişimini dinle

    tk.Label(filter_frame, text="Seri:", bg="#F5F5F5").pack()
    seri_combobox = ttk.Combobox(filter_frame, values=[])  # Dinamik olarak güncellenecek
    seri_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Model:", bg="#F5F5F5").pack()
    model_combobox = ttk.Combobox(filter_frame, values=[])  # Dinamik olarak güncellenecek
    model_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Yıl:", bg="#F5F5F5").pack()
    yil_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Yıl"]) for car in car_data_records if "Yıl" in car and pd.notnull(car["Yıl"])))
    )
    yil_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Renk:", bg="#F5F5F5").pack()
    renk_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Renk"]) for car in car_data_records if "Renk" in car and pd.notnull(car["Renk"])))
    )
    renk_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Kasa Tipi:", bg="#F5F5F5").pack()
    kasa_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Kasa Tipi"]) for car in car_data_records if "Kasa Tipi" in car and pd.notnull(car["Kasa Tipi"])))
    )
    kasa_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Yakıt Tipi:", bg="#F5F5F5").pack()
    yakit_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Yakıt Tipi"]) for car in car_data_records if "Yakıt Tipi" in car and pd.notnull(car["Yakıt Tipi"])))
    )
    yakit_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Vites Tipi:", bg="#F5F5F5").pack()
    vites_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Vites Tipi"]) for car in car_data_records if "Vites Tipi" in car and pd.notnull(car["Vites Tipi"])))
    )
    vites_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Kimden:", bg="#F5F5F5").pack()
    kimden_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Kimden"]) for car in car_data_records if "Kimden" in car and pd.notnull(car["Kimden"])))
    )
    kimden_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Min Fiyat:", bg="#F5F5F5").pack()
    min_fiyat_entry = tk.Entry(filter_frame)
    min_fiyat_entry.pack(pady=2)

    tk.Label(filter_frame, text="Max Fiyat:", bg="#F5F5F5").pack()
    max_fiyat_entry = tk.Entry(filter_frame)
    max_fiyat_entry.pack(pady=2)

    tk.Button(filter_frame, text="Uygula", command=apply_filter, bg="#1E90FF", fg="white").pack(pady=10)
    tk.Button(filter_frame, text="Temizle", command=clear_filters, bg="#1E90FF", fg="white").pack(pady=5)  # Temizle buton

    tk.Button(result_frame, text="Show Valuation History", command=show_valuation_history, bg="#1E90FF", fg="white").pack(pady=10)

    # Sonuç alanı
    search_results = tk.Text(result_frame, wrap=tk.WORD, width=80, height=30, bg="white", fg="black")
    search_results.pack(padx=10, pady=10)

    # "Save Filter" ve "Load Filter" butonlarını sonuçların altına al
    tk.Button(result_frame, text="Save Filter", command=save_filters_with_name, bg="#1E90FF", fg="white").pack(pady=5)
    tk.Button(result_frame, text="Load Filter", command=load_filter, bg="#1E90FF", fg="white").pack(pady=5)
    
    # Seri değiştirildiğinde modelleri güncelleme
    def update_models(event):
        selected_seri = seri_combobox.get().lower()
        related_models = sorted(set(
            str(car["Model"]) for car in car_data_records
            if "Model" in car and "Seri" in car and pd.notnull(car["Model"]) and pd.notnull(car["Seri"]) and selected_seri in car["Seri"].lower()
        ))
        model_combobox["values"] = related_models

    # Seri combobox'ına model güncelleme eventi bağlama
    seri_combobox.bind("<<ComboboxSelected>>", update_models)

def user_panel():
    clear_frame()
    add_home_button(root)  # Sol üst köşeye anasayfa butonu ekle
    tk.Label(root, text="User Login", font=("Arial", 14)).pack(pady=10)

    tk.Label(root, text="Username:").pack()
    username_entry = tk.Entry(root)
    username_entry.pack()

    tk.Label(root, text="Password:").pack()
    password_entry = tk.Entry(root, show="*")
    password_entry.pack()

    def user_login():
        username = username_entry.get()
        password = password_entry.get()
        user = next((user for user in user_database if user["username"] == username and user["password"] == password), None)
        if user:
            user_main_menu()
        else:
            tk.Label(root, text="Invalid credentials.", fg="red").pack()

    tk.Button(root, text="Login", command=user_login).pack(pady=5)
    tk.Button(root, text="Register", command=register_panel).pack(pady=5)

    # Admin Girişi Paneli
def admin_panel():
    clear_frame()
    add_home_button(root)  # Sol üst köşeye anasayfa butonu ekle
    tk.Label(root, text="Admin Login", font=("Arial", 14), bg="#1E90FF", fg="white").pack(pady=10)

    tk.Label(root, text="Admin Name:", bg="#1E90FF", fg="white").pack()
    admin_name_entry = tk.Entry(root)
    admin_name_entry.pack()

    tk.Label(root, text="Password:", bg="#1E90FF", fg="white").pack()
    admin_password_entry = tk.Entry(root, show="*")
    admin_password_entry.pack()

    def admin_login():
        admin_name = admin_name_entry.get()
        admin_password = admin_password_entry.get()
        if admin_name == ADMIN_CREDENTIALS["username"] and admin_password == ADMIN_CREDENTIALS["password"]:
            admin_main_menu()
        else:
            tk.Label(root, text="Invalid admin credentials.", fg="red").pack()

    tk.Button(root, text="Login", command=admin_login, bg="green", fg="white").pack(pady=5)

# Kayıt Paneli
def register_panel():
    clear_frame()
    add_home_button(root)  # Sol üst köşeye anasayfa butonu ekle
    tk.Label(root, text="Register", font=("Arial", 14), bg="#1E90FF", fg="white").pack(pady=10)

    tk.Label(root, text="Username:", bg="#1E90FF", fg="white").pack()
    username_entry = tk.Entry(root)
    username_entry.pack()

    tk.Label(root, text="Name Surname:", bg="#1E90FF", fg="white").pack()
    name_surname_entry = tk.Entry(root)
    name_surname_entry.pack()

    tk.Label(root, text="Password:", bg="#1E90FF", fg="white").pack()
    password_entry = tk.Entry(root, show="*")
    password_entry.pack()

    tk.Label(root, text="Confirm Password:", bg="#1E90FF", fg="white").pack()
    confirm_password_entry = tk.Entry(root, show="*")
    confirm_password_entry.pack()

    tk.Label(root, text="Email:", bg="#1E90FF", fg="white").pack()
    email_entry = tk.Entry(root)
    email_entry.pack()

    def register_user():
        username = username_entry.get()
        name_surname = name_surname_entry.get()
        password = password_entry.get()
        confirm_password = confirm_password_entry.get()
        email = email_entry.get()

        if any(user["username"] == username for user in user_database):
            tk.Label(root, text="Username already exists.", fg="red").pack()
            return

        if password != confirm_password:
            tk.Label(root, text="Passwords do not match.", fg="red").pack()
            return

        user_database.append({"username": username, "name_surname": name_surname, "password": password, "email": email})
        save_user_database()  # Kullanıcı kayıtlarını dosyaya kaydet
        tk.Label(root, text="Registration successful!", fg="green").pack()
        choose_role_panel()

    tk.Button(root, text="Sign Up", command=register_user, bg="green", fg="white").pack(pady=5)
    tk.Button(root, text="Cancel", command=choose_role_panel, bg="red", fg="white").pack(pady=5)

# Kullanıcı Ana Menü
def user_main_menu():
    clear_frame()
    tk.Label(root, text="Car Search and Filter", font=("Arial", 14), bg="#1E90FF", fg="white").pack(pady=10)
    add_home_button(root)  # Sol üst köşeye anasayfa butonu ekle

    global main_frame, marka_combobox, seri_combobox, model_combobox, yil_combobox, renk_combobox, kasa_combobox, sehir_combobox, yakit_combobox, vites_combobox, kimden_combobox, min_fiyat_entry, max_fiyat_entry, search_results

    # Ana çerçeve
    main_frame = tk.Frame(root, bg="#F5F5F5")
    main_frame.pack(fill=tk.BOTH, expand=True)

    # Sol panel (filtreleme)
    filter_frame = tk.Frame(main_frame, padx=10, pady=10, bg="#F5F5F5")
    filter_frame.pack(side=tk.LEFT, fill=tk.Y)

    # Sağ panel (sonuçlar)
    result_frame = tk.Frame(main_frame, padx=10, pady=10, bg="#F5F5F5")
    result_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

    # Filtreleme alanları
    filter_label = tk.Label(filter_frame, text="Filters", font=("Arial", 14), bg="#F5F5F5")
    filter_label.pack(pady=5)

    tk.Label(filter_frame, text="Marka:", bg="#F5F5F5").pack()
    marka_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Marka"]) for car in car_data_records if "Marka" in car and pd.notnull(car["Marka"])))
    )
    marka_combobox.pack(pady=2)
    marka_combobox.bind("<<ComboboxSelected>>", update_series)  # Marka değişimini dinle

    tk.Label(filter_frame, text="Seri:", bg="#F5F5F5").pack()
    seri_combobox = ttk.Combobox(filter_frame, values=[])  # Dinamik olarak güncellenecek
    seri_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Model:", bg="#F5F5F5").pack()
    model_combobox = ttk.Combobox(filter_frame, values=[])  # Dinamik olarak güncellenecek
    model_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Yıl:", bg="#F5F5F5").pack()
    yil_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Yıl"]) for car in car_data_records if "Yıl" in car and pd.notnull(car["Yıl"])))
    )
    yil_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Renk:", bg="#F5F5F5").pack()
    renk_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Renk"]) for car in car_data_records if "Renk" in car and pd.notnull(car["Renk"])))
    )
    renk_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Kasa Tipi:", bg="#F5F5F5").pack()
    kasa_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Kasa Tipi"]) for car in car_data_records if "Kasa Tipi" in car and pd.notnull(car["Kasa Tipi"])))
    )
    kasa_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Yakıt Tipi:", bg="#F5F5F5").pack()
    yakit_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Yakıt Tipi"]) for car in car_data_records if "Yakıt Tipi" in car and pd.notnull(car["Yakıt Tipi"])))
    )
    yakit_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Vites Tipi:", bg="#F5F5F5").pack()
    vites_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Vites Tipi"]) for car in car_data_records if "Vites Tipi" in car and pd.notnull(car["Vites Tipi"])))
    )
    vites_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Kimden:", bg="#F5F5F5").pack()
    kimden_combobox = ttk.Combobox(
        filter_frame, 
        values=sorted(set(str(car["Kimden"]) for car in car_data_records if "Kimden" in car and pd.notnull(car["Kimden"])))
    )
    kimden_combobox.pack(pady=2)

    tk.Label(filter_frame, text="Min Fiyat:", bg="#F5F5F5").pack()
    min_fiyat_entry = tk.Entry(filter_frame)
    min_fiyat_entry.pack(pady=2)

    tk.Label(filter_frame, text="Max Fiyat:", bg="#F5F5F5").pack()
    max_fiyat_entry = tk.Entry(filter_frame)
    max_fiyat_entry.pack(pady=2)

    tk.Button(filter_frame, text="Uygula", command=apply_filter, bg="green", fg="white").pack(pady=10)
    tk.Button(filter_frame, text="Temizle", command=clear_filters, bg="red", fg="white").pack(pady=5)  # Temizle buton
    tk.Button(result_frame, text="Show Valuation History", command=show_valuation_history, bg="blue", fg="white").pack(pady=10)

    # Sonuç alanı
    search_results = tk.Text(result_frame, wrap=tk.WORD, width=80, height=30, bg="white", fg="black")
    search_results.pack(padx=10, pady=10)

    # "Save Filter" ve "Load Filter" butonlarını sonuç alanının altına ekle
    tk.Button(result_frame, text="Save Filter", command=save_filters_with_name, bg="blue", fg="white").pack(pady=5)
    tk.Button(result_frame, text="Load Filter", command=load_filter, bg="blue", fg="white").pack(pady=5)

    # Seri değiştirildiğinde modelleri güncelleme
    def update_models(event):
        selected_seri = seri_combobox.get().lower()
        related_models = sorted(set(
            str(car["Model"]) for car in car_data_records
            if "Model" in car and "Seri" in car and pd.notnull(car["Model"]) and pd.notnull(car["Seri"]) and selected_seri in car["Seri"].lower()
        ))
        model_combobox["values"] = related_models

    # Seri combobox'ına model güncelleme eventi bağlama
    seri_combobox.bind("<<ComboboxSelected>>", update_models)

def user_panel():
    clear_frame()
    add_home_button(root)  # Sol üst köşeye anasayfa butonu ekle
    tk.Label(root, text="User Login", font=("Arial", 14), bg="#1E90FF", fg="white").pack(pady=10)

    tk.Label(root, text="Username:", bg="#1E90FF", fg="white").pack()
    username_entry = tk.Entry(root)
    username_entry.pack()

    tk.Label(root, text="Password:", bg="#1E90FF", fg="white").pack()
    password_entry = tk.Entry(root, show="*")
    password_entry.pack()

    def user_login():
        username = username_entry.get()
        password = password_entry.get()
        user = next((user for user in user_database if user["username"] == username and user["password"] == password), None)
        if user:
            user_main_menu()
        else:
            tk.Label(root, text="Invalid credentials.", fg="red").pack()

    tk.Button(root, text="Login", command=user_login, bg="green", fg="white").pack(pady=5)
    tk.Button(root, text="Register", command=register_panel, bg="blue", fg="white").pack(pady=5)

# Admin Ana Menü
def admin_main_menu():
    clear_frame()
    tk.Label(root, text="Admin Panel", font=("Arial", 14)).pack(pady=10)
    add_home_button(root)  # Sol üst köşeye anasayfa butonu ekle

    tk.Button(root, text="Add Car", command=add_car_panel).pack(pady=5)
    tk.Button(root, text="Logout", command=choose_role_panel).pack(pady=5)

# Admin Araç Ekleme Paneli
def add_car_panel():
    clear_frame()
    tk.Label(root, text="Add New Car", font=("Arial", 14), bg="#1E90FF", fg="white").pack(pady=10)
    add_home_button(root)  # Sol üst köşeye anasayfa butonu ekle

    tk.Label(root, text="Marka:", bg="#1E90FF", fg="white").pack()
    marka_combobox = ttk.Combobox(root, values=["Audi", "BMW", "Chevrolet", "Citroen", "Dacia", "Fiat", "Ford", "Honda", "Hyundai", "Wolkwagen", "Skoda", "Toyota", "Nissan", "Mazda", "Renault", "Volvo"])
    marka_combobox.pack()

    tk.Label(root, text="Seri:", bg="#1E90FF", fg="white").pack()
    seri_entry = tk.Entry(root)
    seri_entry.pack()

    tk.Label(root, text="Model:", bg="#1E90FF", fg="white").pack()
    model_entry = tk.Entry(root)
    model_entry.pack()

    tk.Label(root, text="Yıl:", bg="#1E90FF", fg="white").pack()
    yil_entry = tk.Entry(root)
    yil_entry.pack()

    tk.Label(root, text="Renk:", bg="#1E90FF", fg="white").pack()
    renk_entry = tk.Entry(root)
    renk_entry.pack()

    tk.Label(root, text="Kasa Tipi:", bg="#1E90FF", fg="white").pack()
    kasa_combobox = ttk.Combobox(root, values=["Sedan", "Hatchback/3", "SUV", "Coupe", "Cabrio", "MPV", "Hatchback/5", "Pick-up", "Roadster", "Station wagon"])  # Sabit değerler
    kasa_combobox.pack()

    tk.Label(root, text="Yakıt Tipi:", bg="#1E90FF", fg="white").pack()
    yakit_combobox = ttk.Combobox(root, values=["Benzin", "Dizel", "LPG", "Elektrik"])  # Sabit değerler
    yakit_combobox.pack()

    tk.Label(root, text="Vites Tipi:", bg="#1E90FF", fg="white").pack()
    vites_combobox = ttk.Combobox(root, values=["Manuel", "Otomatik", "Yarı Otomatik"])  # Sabit değerler
    vites_combobox.pack()

    tk.Label(root, text="Kimden:", bg="#1E90FF", fg="white").pack()
    kimden_combobox = ttk.Combobox(root, values=["Sahibinden", "Galeriden", "Yetkili Bayiden"])  # Sabit değerler
    kimden_combobox.pack()

    tk.Label(root, text="Fiyat:", bg="#1E90FF", fg="white").pack()
    fiyat_entry = tk.Entry(root)
    fiyat_entry.pack()

    def save_car():
        new_car = {
            "Marka": marka_combobox.get(),
            "Seri": seri_entry.get(),
            "Model": model_entry.get(),
            "Yıl": yil_entry.get(),
            "Renk": renk_entry.get(),
            "Kasa Tipi": kasa_combobox.get(),
            "Yakıt Tipi": yakit_combobox.get(),
            "Vites Tipi": vites_combobox.get(),
            "Kimden": kimden_combobox.get(),
            "Fiyat": fiyat_entry.get()
        }

        # Araç verilerini ekle ve CSV dosyasına kaydet
        car_data_records.append(new_car)
        car_data_df = pd.DataFrame(car_data_records)
        car_data_df.to_csv(file_path, index=False)

        tk.messagebox.showinfo("Success", "Car added successfully!")
        admin_main_menu()

    tk.Button(root, text="Save", command=save_car, bg="green", fg="white").pack(pady=5)
    tk.Button(root, text="Cancel", command=admin_main_menu, bg="red", fg="white").pack(pady=5)


# Ana sayfa butonu fonksiyonu
def go_home():
    choose_role_panel()

def add_home_button(frame):
    tk.Button(frame, text="Anasayfa", command=go_home, bg="#FFD700", fg="black").pack(anchor='nw')

def clear_frame():
    for widget in root.winfo_children():
        widget.destroy()

def choose_role_panel():
    clear_frame()
    tk.Label(root, text="Choose Role", font=("Arial", 14), bg="#1E90FF", fg="white").pack(pady=10)

    tk.Button(root, text="Admin Login", command=admin_panel, bg="green", fg="white").pack(pady=5)
    tk.Button(root, text="User Login", command=user_panel, bg="blue", fg="white").pack(pady=5)
    tk.Button(root, text="Register", command=register_panel, bg="orange", fg="white").pack(pady=5)

# Uygulama başlatma
root = tk.Tk()
root.title("Car Evaluation Tool")
root.geometry("900x900")
root.configure(bg="#F5F5F5")

choose_role_panel()
root.mainloop()